In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)   

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

np.set_printoptions(threshold=np.inf)   #affichage non tronqué

## recupere BDD -> Matrices d'inputs

In [2]:
import pandas as pd
ROWS=671

#recupere les données du fichier csv et les mets dans les ndarray X et Y (grace à to_numpy(),sinon ce sont des types dataframe)
#Ils ont donc une shape de (671,1)

X= pd.read_csv("sentence.csv", encoding = "utf-8",sep=";",usecols = ["sen_sentence"],nrows=ROWS).to_numpy()
Y= pd.read_csv("sentence.csv", encoding = "utf-8",sep=";",usecols = ["sen_en_full"],nrows=ROWS).to_numpy().reshape(-1,)

In [3]:
import pprint                            #permet un affichage ligne par ligne,équivalent au println() de java
from nltk.tag import StanfordPOSTagger   #librairie utilisée pour tagger

jar = './stanford-postagger-full-2018-10-16/stanford-postagger.jar'    # recupere les modeles telechargés dans le fichier
model = './stanford-postagger-full-2018-10-16/models/french.tagger'
java_path = "C:/Program Files/Java/jre1.8.0_211/bin/java.exe"          # on s'assure du lien pour java
os.environ['JAVAHOME'] = java_path                                     # variable d'environnement

pos_tagger = StanfordPOSTagger(model, jar, encoding='utf-8' )          #prêt à être utilisé

In [4]:
#fonctions qui transforme une phrase en matrice lignes (taille 28) de nos inputs , binarisation des tags

def senToTags(sentence):
    sen_tags=[m[1] for m in pos_tagger.tag(sentence)]        #tout les tags presents dans la phrase
    #dictionnaire des tags possibles (28)
    tags=['P', 'N',  'ADJ', 'NC', 'DET', 'NPP', 'V', 'VPP', 'ADV', 'PROREL', 'CLS', 'VINF', 'CC', 'PUNC', 'PRO', 'ET', 'CS', 'CLR', 'CLO', 'VPR', 'ADVWH', 'C', 'VIMP', 'CL', 'VS', 'PROWH', 'ADJWH', 'PREF']
    result=list()
    for tag in tags:            # pour chaque tag du dictionnaire
        if tag in sen_tags:     # on vérifie s'il est présent dans la phrase
            result.append(1)
        else:
            result.append(0)
    return result

In [5]:
print("exemple : senToTags(\" Les Americains ont réagi en six semaines là où il a fallu neuf mois aux Francais.\") ---> ",senToTags("Les Americains ont réagi en six semaines là où il a fallu neuf mois aux Francais."))

exemple : senToTags(" Les Americains ont réagi en six semaines là où il a fallu neuf mois aux Francais.") --->  [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
import time
start = time.time()          # recupere l'heure exacte pour calculer le temps d'execution
X_tmp=list()                 #initialise une list
for i in range(len(X)):
    X_tmp+=senToTags(X[i])  #on applique cette fonction (cellule precedente) à chaque phrase, à chaque ligne de X donc
end = time.time()
print("time execution : ",end - start)

time execution :  1425.9014670848846


In [7]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X_tmp=np.asarray(X_tmp).reshape(-1,28)  #redimensionne en imposant 28 colonnes , peut importe le nb de lignes
#simple separation de X_tmp en X_train et X_test selon le ratio , test_size ... meme chose pour Y
# le random_state=2 permet uniquement de fixer l'aleatoire apres un premier tirage pour ne pas avoir de problemes en reexecutant 
X_train, X_test, Y_train, Y_test = train_test_split(X_tmp, Y, test_size=0.1, random_state=2)

print("Before OverSampling, counts of label '1': {}".format(sum(Y_train==1)))    # nb de label=1 avant equilibrage
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train==0))) # nb de label=0 avant equilibrage


sm = SMOTE(random_state=2)                       #initialise le regulateur qui va en realité crée des doublons de la classe
X_train, Y_train = sm.fit_sample(X_train, Y_train)  # inferieur pour egaliser le nb de labels

print('After OverSampling, the shape of train_X: {}'.format(X_train.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(Y_train.shape)) 
print("After OverSampling, counts of label '1': {}".format(sum(Y_train == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(Y_train == 0))) 

Before OverSampling, counts of label '1': 346
Before OverSampling, counts of label '0': 257 

After OverSampling, the shape of train_X: (692, 28)
After OverSampling, the shape of train_y: (692,) 

After OverSampling, counts of label '1': 346
After OverSampling, counts of label '0': 346


# Forward Neural Network

### Fonctions pour le reseau de neurones

In [8]:
#couche de neurones, on aurait pu utilisé tf.layers.dense() qui prend les meme parametres et qui crée une couche intégralement connectée, elle se charge de creer les var necessaires en utilisant la stratégie d'initialisationj appropriée

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name): #portée de nom , facultatif, pour mieux retrouver
        n_inputs = int(X.shape[1])  #nb entrées = nb de colonnes de X donc 28 
        
        #ces 3 lignes créent la var W qui contiendra la matrice des poids des connexions entre chaque entrée et chaque neurone
        stddev = 2 / np.sqrt(n_inputs)  #facultatif, on tronquera avec cet ecart-type qui permet à l'algo de converger plus rapidement,plus stddev est grand et plus l'ecart sera grand
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev) #matrice n_inputs x n_neurons valeurs proche de 0 , tronquées avec stddeb   
        W = tf.Variable(init,name="kernel")  #weights random
        
        b = tf.Variable(tf.zeros([n_neurons]), name="bias") #biais,initialisé à 0
        Z = tf.matmul(X, W) + b    #z=X.W +b
        tf.cast(Z,tf.int32)
        if activation is not None:  #passe par une fonction d'activation si elle existe
            return activation(Z)
        else:
            return Z

        
        
        
#return un Y avec une dimension = valeur max+1 , donc ici y_one_hot aura une shape=(any,2)
#voir exemple cellule d'apres

def to_one_hot(y):
    n_classes = y.max() + 1  # dimension +1 dans notre cas , ce sera forcement 2
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes)) #matrice de dimension(m,n_classes) remplit de 0
    Y_one_hot[np.arange(m), y] = 1  #on mets un 1 dans la colonne 1 etc.. voir exemple
    return Y_one_hot



#Homogeneisation, decoupage de X et Y en paquets de taille = batch_size, choisis aléatoirement dans X et Y , permets d'eviter

def shuffle_batch(X, y, batch_size):               #de calculer que sur une série de données "particuliere" comme l'interview de Rihanna et pas sur le reste, ici au moins , on a des données homogenes
    rnd_idx = np.random.permutation(len(X))              #array de taille = 671  de valeur aleatoires < 671  
    n_batches = len(X) // batch_size               #nb de paquets
    for batch_idx in np.array_split(rnd_idx, n_batches):  #np.array_split(X,n) split X en n parties(array)
        X_batch, y_batch = X[batch_idx], y[batch_idx]     #on declare nos paquets 
        yield X_batch, y_batch                           #yield est comme un return sauf que c'est un generateur , ce qui permet de gagner en memoire , il ne parcourt pas à chaque appel toute la fonction mais recupere à la volée la valeur puis incremente ensuite le generateur , un lien qui explique davanatage : https://www.journaldunet.fr/web-tech/developpement/1202863-a-quoi-sert-le-mot-cle-yield-en-python/

### exemples fonctions utilisées

In [9]:
#exemple y_one_hot

y=np.asarray([1,0,1,1,0,0])

print("y_shape = ",y.shape)
y1=to_one_hot(y)
print("y1_shape = ",y1.shape)

#on mets pour chaque element de y ayant pour valeur i , un 1 à la colonne i et sur le reste de la ligne, que des 0 
print("y1 =" ,y1)

y_shape =  (6,)
y1_shape =  (6, 2)
y1 = [[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [10]:
#exemple truncated_normal

import tensorflow as tf
with  tf.Session() as sess:
    p=tf.truncated_normal((3,2),stddev=0.2).eval()
    print(p)

[[ 0.25330153  0.15354085]
 [ 0.03800653  0.3326441 ]
 [-0.09796198  0.03484468]]


In [11]:
# exemple argmax
a = np.arange(6).reshape(2,3) + 10
print("a =",a)
print("axis=0 ->",np.argmax(a,axis=0))  #axis=0 prends pour chaque colonne,l'indice du max
print("axis=1 ->",np.argmax(a,axis=1))  #axis=1 prends pour chaque ligne,l'indice du max 

a = [[10 11 12]
 [13 14 15]]
axis=0 -> [1 1 1]
axis=1 -> [2 2]


# 28_18_10_2 reseau  0.01 learning rate,75 epochs,26 batchs -> 88%

In [12]:
import tensorflow as tf
import sklearn
n_inputs = 28    # effet entonnoir , quelques liens pour comprendre comment choisir 
n_hidden1 = 18   # le nb de neurons par couches : https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
n_hidden2 = 10   # https://www.researchgate.net/post/How_to_decide_the_number_of_hidden_layers_and_nodes_in_a_hidden_layer
n_outputs = 2    # 2 classes , 2 outputs

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")   #permets d'utiliser X sans lui rentrer tout de suite ses donnees
y = tf.placeholder(tf.int32, shape=(None), name="y")   #None => any


hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",     #prend en entrée nos données(phrase), puisque X sera nourrit dans le feed_dict par X_train
                               activation=tf.nn.relu)       #et en sortie le nb de neurones dans la 2e couche
hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", # prend en entrée la couche precedente etc...
                               activation=tf.nn.relu)
logits = tf.layers.dense(hidden2, n_outputs, name="outputs")  #derniere couche, donc renvoie les valeurs sorties par le reseau

learning_rate =0.01    


#le reseau est pret , on definit la fonction de cout, et pour ca on utilise l'entropie croisée
xentropy = tf.keras.backend.binary_crossentropy(tf.cast(y,tf.float32),logits)  # penalise les faible proba , on utilise une pour binary classificaion ici 
loss = tf.reduce_mean(xentropy)                                                #  l'entroprie est calculée à partir des logits et attend les labels de y , reduce_mean est l'entroprie croisée moyenne sur toutes les instances

#ajuste les parametres du modele pour minimiser la fonction cout
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

#comment on évalue le modele,ici l'exactitude , on va verifier si la prevision est correcte en verifiant si le logit le plus eleve = le bon label
correct = tf.nn.in_top_k(logits, tf.argmax(y, 1), 1) #voir exemple dans la cellule d'apres pour cette fonction
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))  #moyenne , donc precision gloable du modele

init = tf.global_variables_initializer()  #noeud qui initialise toutes les variables
saver = tf.train.Saver()                  #pour enregistrer le modele
n_epochs = 75                           
batch_size = 26

#phase d'execution
with tf.Session() as sess:
    init.run()                            #execute le noeud init
    
    for epoch in range(n_epochs):         # a chaque epoch, on evalue chaque  minis paquets qui correspondent au X_train a la fin de la sous boucle
        for X_batch, y_batch in shuffle_batch(X_train, Y_train, batch_size):  #pour chaque mini-lot choisis de maniere homogene dans X_train,Y_train
                _, cost, corr, acc = sess.run([training_op, loss, correct, accuracy], feed_dict={X: X_train, y: to_one_hot(Y_train)})  #on execute l'operation d'entrainement sur training_op et sur d'autres parametres pour avoir une meilleure idée de ce qu'il se passe (cf print suivant),
        
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: to_one_hot(y_batch)})               #a la fin de chaque etape, le code evalue l'exactitude du modele sur le dernier mini lot (car hors boucle) 
        print(epoch,'Batch accuracy: {} Loss: {} Train_Accuracy: {}'.format(acc_batch,cost, acc))
    save_path = saver.save(sess, "./my_model_final.ckpt")

#le modele est entrainé, on test
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")  # restore le model
    Z = logits.eval(feed_dict={X: X_test})        # evalue la couche de sortie sur X_test
    y_pred = np.argmax(Z, axis=1)                 #renvoie l'indice du max de chaque ligne (axis=1) de Z , donc la proba la plus elevée parmis les classes
    
    print("Predicted classes:",y_pred)
    print("Actual classes:   ", Y_test)
    print("accuracy_score : ",sklearn.metrics.accuracy_score(Y_test, y_pred, normalize=True ,sample_weight=None))
    print("matthews_corrcoef : ",sklearn.metrics.matthews_corrcoef(Y_test, y_pred, sample_weight=None))
    print("F1_score : ",sklearn.metrics.f1_score(Y_test, y_pred, labels=None, pos_label=1, average='binary', sample_weight=None))
    

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0 Batch accuracy: 0.692307710647583 Loss: 0.7229865193367004 Train_Accuracy: 0.5809248685836792
1 Batch accuracy: 0.692307710647583 Loss: 0.6658163666725159 Train_Accuracy: 0.6315028667449951
2 Batch accuracy: 0.7692307829856873 Loss: 0.6371657252311707 Train_Accuracy: 0.6791907548904419
3 Batch accuracy: 0.8461538553237915 Loss: 0.612127959728241 Train_Accuracy: 0.7124277353286743
4 Batch accuracy: 0.807692289352417 Loss: 0.5889171361923218 Train_Accuracy: 0.7413294911384583
5 Batch accuracy: 0.692307710647583 Loss: 0.5644799470901489 Train_Accuracy: 0.7687861323356628
6 Batch accuracy: 0.7307692170143127 Loss: 0.5397977232933044 Train_Accuracy: 0.7846820950508118
7 Batch accuracy: 0.6538461446762085 Loss: 0.5176363587379456 Train_Accuracy: 0.7991329431533813
8 Batch accuracy: 0.8846153616905212 Loss: 0.49833

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
Predicted classes: [1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1
 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0]
Actual classes:    [1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 1
 1 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0]
accuracy_score :  0.8823529411764706
matthews_corrcoef :  0.7628596338273758
F1_score :  0.8709677419354839


### exemple de tf.nn.in_top_k(logits, y, k)

In [13]:
# logits est un tenseur de type float32 de shape (batch_size,n_classes)
# y est un tenseur de type int32 de shape (batch_size) , c'est les classes_ids
# k est le nb d'elements à prendre (precision)
#retourne un array de booleen
test = tf.nn.in_top_k([[0,1], [1,0], [0,1], [1, 0], [0, 1]], [0, 1, 1, 1, 1], 1) #logits_shape=(5,2) et y_shape=(5,)
with tf.Session() as sess:
    print("retourne le booleen qui dit si le max de logits[i] se trouve effectivement à colonne y[i]")
    print(" ","ici, est ce que le max de logits[0] = [0,1] se trouve à la colonne y[0]=0 ? --> False")
    print(" ","ici, est ce que le max de logits[2] = [0,1] se trouve à la colonne y[2]=1 ? --> true")
    print()
    print(test.eval())


retourne le booleen qui dit si le max de logits[i] se trouve effectivement à colonne y[i]
  ici, est ce que le max de logits[0] = [0,1] se trouve à la colonne y[0]=0 ? --> False
  ici, est ce que le max de logits[2] = [0,1] se trouve à la colonne y[2]=1 ? --> true

[False False  True False  True]
